In [3]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark
!pip install py4j

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 229 kB in 1s (179 kB/s)
Reading package lists... Done


In [4]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [9]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv("file://" + SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)



In [10]:
# 2. Create a temporary view of the DataFrame.

home_sales_df.createOrReplaceTempView("home_sales")


In [12]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
avg_price_per_year_4_bed = spark.sql("""
    SELECT year(date) as year, ROUND(AVG(price), 2) as avg_price
    FROM home_sales
    WHERE bedrooms = 4
    GROUP BY year(date)
    ORDER BY year
""")
avg_price_per_year_4_bed.show()

+----+---------+
|year|avg_price|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [15]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
avg_price_per_year_built_3_bed_3_bath = spark.sql("""
    SELECT year(date) as year, ROUND(AVG(price), 2) as avg_price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3
    GROUP BY year(date)
    ORDER BY year
""")
avg_price_per_year_built_3_bed_3_bath.show()

+----+---------+
|year|avg_price|
+----+---------+
|2019|287287.82|
|2020|294204.16|
|2021|294211.46|
|2022|292725.69|
+----+---------+



In [16]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?

avg_price_per_year_specific_criteria = spark.sql("""
    SELECT year(date) as year, ROUND(AVG(price), 2) as avg_price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
    GROUP BY year(date)
    ORDER BY year
""")
avg_price_per_year_specific_criteria.show()


+----+---------+
|year|avg_price|
+----+---------+
|2019|289859.14|
|2020|292289.09|
|2021|296330.96|
|2022|290242.99|
+----+---------+



In [17]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()
avg_price_per_view_over_350k = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) as avg_price
    FROM home_sales
    WHERE price >= 350000
    GROUP BY view
""")
end_time = time.time()
run_time = round(end_time - start_time, 2)
avg_price_per_view_over_350k.show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|  31| 399856.95|
|  85|1056336.74|
|  65| 736679.93|
|  53|  755214.8|
|  78|1080649.37|
|  34| 401419.75|
|  81|1053472.79|
|  28| 402124.62|
|  76|1058802.78|
|  26| 401506.97|
|  27| 399537.66|
|  44| 400598.05|
|  12| 401501.32|
|  91|1137372.73|
|  22| 402022.68|
|  93|1026006.06|
|  47|  398447.5|
|   1| 401044.25|
|  52| 733780.26|
|  13| 398917.98|
+----+----------+
only showing top 20 rows

--- 1.0172827243804932 seconds ---


In [18]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_sales")

In [19]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [21]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

avg_price_per_view_over_350k_cached = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) as avg_price
    FROM home_sales
    WHERE price >= 350000
    GROUP BY view
""")
end_time_cached = time.time()
run_time_cached = round(end_time_cached - start_time, 2)
avg_price_per_view_over_350k_cached.show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+----------+
|view| avg_price|
+----+----------+
|  31| 399856.95|
|  85|1056336.74|
|  65| 736679.93|
|  53|  755214.8|
|  78|1080649.37|
|  34| 401419.75|
|  81|1053472.79|
|  28| 402124.62|
|  76|1058802.78|
|  26| 401506.97|
|  27| 399537.66|
|  44| 400598.05|
|  12| 401501.32|
|  91|1137372.73|
|  22| 402022.68|
|  93|1026006.06|
|  47|  398447.5|
|   1| 401044.25|
|  52| 733780.26|
|  13| 398917.98|
+----+----------+
only showing top 20 rows

--- 1.8409576416015625 seconds ---


In [22]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales_df.write.partitionBy("date").parquet("formatted_home_sales.parquet")


In [23]:
# 11. Read the parquet formatted data.
formatted_home_sales_df = spark.read.parquet("formatted_home_sales.parquet")


In [24]:
# 12. Create a temporary table for the parquet data.
formatted_home_sales_df.createOrReplaceTempView("formatted_home_sales")


In [25]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

avg_price_per_view_over_350k_parquet = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) as avg_price
    FROM formatted_home_sales
    WHERE price >= 350000
    GROUP BY view
""")
end_time_parquet = time.time()
run_time_parquet = round(end_time_parquet - start_time, 2)
avg_price_per_view_over_350k_parquet.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|  31| 399856.95|
|  65| 736679.93|
|  78|1080649.37|
|  34| 401419.75|
|  81|1053472.79|
|  28| 402124.62|
|  26| 401506.97|
|  27| 399537.66|
|  44| 400598.05|
|  12| 401501.32|
|  91|1137372.73|
|  22| 402022.68|
|  93|1026006.06|
|  47|  398447.5|
|   1| 401044.25|
|  13| 398917.98|
|   6| 395655.38|
|  16| 399586.53|
|  86|1070444.25|
|   3|  398867.6|
+----+----------+
only showing top 20 rows

--- 8.849273920059204 seconds ---


In [26]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")


In [27]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached("home_sales")

False